# System.Text.Json

System.Text.Json is a highly performant and feature-rich JSON library that is built into .NET Core 3.0 and later. It is the default JSON library for ASP.NET Core 3.0 and later.

This notebook will explore the basic functionality and customization options of System.Text.Json.

See the [System.Text.Json documentation](https://learn.microsoft.com/en-us/dotnet/standard/serialization/system-text-json/overview) for more information.

In [1]:
using System.Text.Json;
using System.Text.Json.Serialization;
using System.Globalization;
using System.Reflection;

## Basic Serialization

Here's a simple example that shows how to serialize an object to JSON.

In [2]:
public class WeatherForecast
{
    public DateTimeOffset Date { get; set; }
    public int TemperatureCelsius { get; set; }
    public string? Summary { get; set; }
    public string? Detail { get; set; }
    public float chanceOfPrecipitation { get; set; }

    public int TemperatureFahrenheit => 32 + (int)(TemperatureCelsius / 0.5556);

    internal int WindSpeed { get; set; } = 35;
    private int Pressure { get; set; } = 120;
}

var weatherForecast = new WeatherForecast
{
    Date = DateTimeOffset.Parse("2019-08-01"),
    TemperatureCelsius = 25,
    Summary = "Hot"
};

string jsonString = JsonSerializer.Serialize(weatherForecast);

Console.WriteLine(jsonString);

{"Date":"2019-08-01T00:00:00-05:00","TemperatureCelsius":25,"Summary":"Hot","Detail":null,"chanceOfPrecipitation":0,"TemperatureFahrenheit":76,"WindSpeed":35}


Here the serializer is using the default serialization options. In particular:
- JSON property names use the same casing as the .NET property names.
- All public properties are serialized, including read-only properties.
- Internal properties are serialized when the serializer is used within the same assembly as the type being serialized.
- Private properties are not serialized.
- Reference properties with `null` values are serialized.

## Basic Deserialization

Here's a simple example that shows how to deserialize JSON to an object.

In [4]:
public class WeatherForecast
{
    public DateTimeOffset Date { get; set; }
    public int TemperatureCelsius { get; set; }
    public string? Summary { get; set; }
}

var jsonString = @"{""Date"":""2019-08-01T00:00:00-07:00"",""temperatureCelsius"":25,""Summary"":""Hot"",""Details"":""Pool weather""}";

var weatherForecast = JsonSerializer.Deserialize<WeatherForecast>(jsonString);

foreach (PropertyInfo pi in typeof(WeatherForecast).GetProperties()) {
    Console.WriteLine($"{pi.Name}: {pi.GetValue(weatherForecast)}");
}

Date: 8/1/2019 12:00:00 AM -07:00
TemperatureCelsius: 0
Summary: Hot


Again, the deserializer is using the default serialization options. In particular:
- JSON property names are treated case-sensitively -- TemperatureCelcius gets the default value of 0 and not 25.
- Unknown JSON properties are ignored -- "Details" is not deserialized.

Another important point is that the deserializer will throw an execption if any of the JSON properties are not valid for the type being deserialized. For example, if the value of the JSON property "TemperatureCelcius" is a string instead of a number, the deserializer throws an exception.

In [16]:
var jsonString = @"{""Date"":""2019-08-01T00:00:00-07:00"",""TemperatureCelsius"":""25"",""Summary"":""Hot""}";

try {
    var weatherForecast = JsonSerializer.Deserialize<WeatherForecast>(jsonString);
}
catch (JsonException e) {
    Console.WriteLine($"Message: {e.Message}");
}

Message: The JSON value could not be converted to System.Int32. Path: $.TemperatureCelsius | LineNumber: 0 | BytePositionInLine: 61.


## Customizing System.Text.Json

The examples above show serialization and deserialization using the default serialization options. However, System.Text.Json lets you customize serialization and deserialization to fit your API conventions. The `JsonSerializerOptions` class has many properties that you can set to customize serialization and deserialization behavior.

### Customizing serialization

Some common serialization customizations are:
- Convert property names to camel case when serializing.
- Don't serialize properties with null values

The following example illustrates these customizations.

In [49]:
public class WeatherForecast
{
    public DateTimeOffset Date { get; set; }
    public int TemperatureCelsius { get; set; }
    public string? Summary { get; set; }
    public string? Detail { get; set; }
}

var weatherForecast = new WeatherForecast
{
    Date = DateTimeOffset.Parse("2019-08-01"),
    TemperatureCelsius = 25,
    Summary = "Hot"
};

var options = new JsonSerializerOptions
{
    // Convert property names to camel case when serializing
    PropertyNamingPolicy = JsonNamingPolicy.CamelCase,
    // Don't serialize properties with null values
    DefaultIgnoreCondition = JsonIgnoreCondition.WhenWritingNull
};
string jsonString = JsonSerializer.Serialize(weatherForecast, options);

Console.WriteLine(jsonString);

{"date":"2019-08-01T00:00:00","temperatureCelsius":25,"summary":"Hot"}


Note that "Detail" is not serialized because it has a null value, and all property names are camel-cased.

### Customizing deserialization

Some common deserialization customizations are:
- allow numbers to be read from JSON strings.
- property names are compared case-insensitively.

The following example illustrates these customizations.

In [50]:
var jsonString = @"{""date"":""2019-08-01T00:00:00-07:00"",""temperaturecelsius"":""25"",""summary"":""Hot""}";
var options = new JsonSerializerOptions
{
    // allow numbers to be read from JSON strings
    NumberHandling = JsonNumberHandling.AllowReadingFromString,
    // property names are compared case-insensitively
    PropertyNameCaseInsensitive = true
};

var weatherForecast = JsonSerializer.Deserialize<WeatherForecast>(jsonString, options);

foreach (PropertyInfo pi in typeof(WeatherForecast).GetProperties()) {
    Console.WriteLine($"{pi.Name}: {pi.GetValue(weatherForecast)}");
}

Date: 8/1/2019 2:00:00 AM
TemperatureCelsius: 25
Summary: Hot
Detail: 


Here the value "25" is deserialized as the number 25, and all properties are recognized even though they are fully lower case.

### Enums as strings

By default, enum values are serialized to the underlying integer value and not the enum name. But System.Text.Json lets you customize this behavior to [serialize the enum value as a string][enums-as-strings].

[enums-as-strings]: https://learn.microsoft.com/en-us/dotnet/standard/serialization/system-text-json/customize-properties?pivots=dotnet-8-0#enums-as-strings

There are a few ways to do this, but I think the best way is adding the `JsonStringEnumConverter` attribute to the enum type.


In [6]:
public class WeatherForecastWithEnum
{
    public DateTimeOffset Date { get; set; }
    public int TemperatureCelsius { get; set; }
    public Summary? Summary { get; set; }
}

[JsonConverter(typeof(JsonStringEnumConverter<Summary>))]
public enum Summary
{
    Cold, Cool, Warm, Hot
}

var weatherForecast = new WeatherForecastWithEnum
{
    Date = DateTimeOffset.Parse("2019-08-01"),
    TemperatureCelsius = 25,
    Summary = Summary.Hot
};

string jsonString = JsonSerializer.Serialize(weatherForecast);

Console.WriteLine(jsonString);

{"Date":"2019-08-01T00:00:00-05:00","TemperatureCelsius":25,"Summary":"Hot"}


What's great about this approach is that applies to both serialization and deserialization.

In [45]:
var jsonString = @"{""Date"":""2019-08-01T00:00:00-07:00"",""TemperatureCelsius"":25,""Summary"":""Hot""}";

var weatherForecast = JsonSerializer.Deserialize<WeatherForecastWithEnum>(jsonString);

foreach (PropertyInfo pi in typeof(WeatherForecastWithEnum).GetProperties()) {
    Console.WriteLine($"{pi.Name}: {pi.GetValue(weatherForecast)}");
}

Date: 8/1/2019 12:00:00 AM -07:00
TemperatureCelsius: 25
Summary: Hot


However, _by default_ enum string values are case-insensitive. 

In [7]:
var jsonString = @"{""Date"":""2019-08-01T00:00:00-07:00"",""TemperatureCelsius"":25,""Summary"":""hOt""}";

var weatherForecast = JsonSerializer.Deserialize<WeatherForecastWithEnum>(jsonString);

foreach (PropertyInfo pi in typeof(WeatherForecastWithEnum).GetProperties()) {
    Console.WriteLine($"{pi.Name}: {pi.GetValue(weatherForecast)}");
}

Date: 8/1/2019 12:00:00 AM -07:00
TemperatureCelsius: 25
Summary: Hot


If you want the deserializer to treat enum string values as case-sensitive, you need to write a custom converter _factory_ -- one of the two converter patterns supported by System.Text.Json (see [this article][custom-converters] for more information).
Here's an example of a custom converter factory that treats enum string values as case-sensitive and how to use it.

[custom-converters]: https://learn.microsoft.com/en-us/dotnet/standard/serialization/system-text-json/converters-how-to

In [7]:
using System.Reflection;
using System.Text.Json;
using System.Text.Json.Serialization;

public class StrictEnumConverterFactory : JsonConverterFactory
{
    public override bool CanConvert(Type typeToConvert) => typeToConvert.IsEnum;

    public override JsonConverter CreateConverter(
        Type type,
        JsonSerializerOptions options)
    {
        JsonConverter converter = (JsonConverter)Activator.CreateInstance(
            typeof(StrictEnumConverterInner<>).MakeGenericType(new Type[] { type }),
            BindingFlags.Instance | BindingFlags.Public,
            binder: null,
            args: new object[] { options },
            culture: null)!;

        return converter;
    }

    private class StrictEnumConverterInner<TEnum> : JsonConverter<TEnum> where TEnum : Enum
    {
        public StrictEnumConverterInner(JsonSerializerOptions options) { }

        public override TEnum Read(
            ref Utf8JsonReader reader,
            Type typeToConvert,
            JsonSerializerOptions options)
        {
            string enumString = reader.GetString();
            if (Enum.TryParse(typeToConvert, enumString, ignoreCase: false, out object value)) {
                return (TEnum) value;
            }
            throw new JsonException($"Unable to convert \"{enumString}\" to Enum \"{typeToConvert}\".");
        }

        public override void Write(
            Utf8JsonWriter writer,
            TEnum enumValue,
            JsonSerializerOptions options) =>
                writer.WriteStringValue(enumValue.ToString());
    }
}

public enum Summary
{
    Cold, Cool, Warm, Hot
}

var jsonString = @"{""Date"":""2019-08-01T00:00:00-07:00"",""TemperatureCelsius"":25,""Summary"":""Hot""}";

var weatherForecast = JsonSerializer.Deserialize<WeatherForecastWithEnum>(jsonString,
    new JsonSerializerOptions {
        Converters = { new StrictEnumConverterFactory() }   // register converter globally -- handles all enums
    }
);

foreach (PropertyInfo pi in typeof(WeatherForecastWithEnum).GetProperties()) {
    Console.WriteLine($"{pi.Name}: {pi.GetValue(weatherForecast)}");
}

var jsonString2 = @"{""Date"":""2019-08-01T00:00:00-07:00"",""TemperatureCelsius"":25,""Summary"":""hot""}";

try {
    var weatherForecast = JsonSerializer.Deserialize<WeatherForecastWithEnum>(jsonString2,
        new JsonSerializerOptions {
            Converters = { new StrictEnumConverterFactory() }   // register converter globally -- handles all enums
        }
    );
}
catch (JsonException e) {
    Console.WriteLine($"JSON exception: {e.Message}");
}


Date: 8/1/2019 12:00:00 AM -07:00
TemperatureCelsius: 25
Summary: Hot
JSON exception: Unable to convert "hot" to Enum "Submission#7+Summary".


### Customizing date-time serialization

System.Text.Json serializes date-time values according to the ISO 8601 standard. But ISO 8601 has many different formats, and you may want to customize the format used by the serializer. You can do that by adding a converter to the `JsonSerializerOptions.Converters` collection.

The following example shows how to specify a converter that translates the date-time value to UTC and then serializes with a format with 3 digits of fractional seconds and uses the "Z" time-zone specifier to indicate UTC.

In [8]:
public class DateTimeOffsetJsonConverter : JsonConverter<DateTimeOffset>
{
    public override DateTimeOffset Read(
        ref Utf8JsonReader reader,
        Type typeToConvert,
        JsonSerializerOptions options) =>
            DateTimeOffset.Parse(reader.GetString()!);

    public override void Write(
        Utf8JsonWriter writer,
        DateTimeOffset dateTimeValue,
        JsonSerializerOptions options) =>
            writer.WriteStringValue(dateTimeValue.UtcDateTime.ToString(
                "yyyy-MM-ddTHH:mm:ss.fffK", CultureInfo.InvariantCulture));
}

public class WeatherForecast
{
    public DateTimeOffset Date { get; set; }
    public int TemperatureCelsius { get; set; }
    public string? Summary { get; set; }
}

var weatherForecast = new WeatherForecast
{
    Date = DateTimeOffset.Parse("2019-08-01"),
    TemperatureCelsius = 25,
    Summary = "Hot"
};

var options = new JsonSerializerOptions
{
    Converters = { new DateTimeOffsetJsonConverter() }
};
string jsonString = JsonSerializer.Serialize(weatherForecast, options);

Console.WriteLine(jsonString);


{"Date":"2019-08-01T05:00:00.000Z","TemperatureCelsius":25,"Summary":"Hot"}


## System.Text.Json in ASP.NET Core applications

When you use System.Text.Json indirectly in an ASP.NET Core app, it is configured with [Web defaults for JsonSerializerOptions](https://learn.microsoft.com/en-us/dotnet/standard/serialization/system-text-json/configure-options#web-defaults-for-jsonserializeroptions), which include:

- property's name are converted to camel case 
- numbers may be read from JSON strings during deserialization.
- property names are compared case-insensitively during deserialization.

All of these customizations were illustrated in the examples above. But suppose you want to change them?


### Configure System.Text.Json in ASP.NET Core

To configure the System.Text.Json options in your app, use the `IServiceCollection.ConfigureHttpJsonOptions` extension method.

The following example shows how to configure the System.Text.Json options used by ASP.NET Core.

```csharp
builder.Services.ConfigureHttpJsonOptions(options =>
{
    // Don't serialize null values
    options.SerializerOptions.DefaultIgnoreCondition = JsonIgnoreCondition.WhenWritingNull;
    // Only read numbers from JSON numbers and always write numbers as JSON numbers (without quotes).
    options.SerializerOptions.NumberHandling = JsonNumberHandling.Strict;
    // Treat property names as case-sensitive in deserialization
    options.SerializerOptions.PropertyNameCaseInsensitive = false;
});
```

## Resources

- [System.Text.Json Overview](https://learn.microsoft.com/en-us/dotnet/standard/serialization/system-text-json/overview)
- [System.Text.Json API Reference](https://learn.microsoft.com/en-us/dotnet/api/system.text.json?view=net-8.0)
